# Building with FireworksAI

This notebook goes through the building blocks to creating magical AI applications with FireworksAI. We will run through the following tasks:
1. Setting up dependencies
2. Calling an LLM and getting a response
3. Calling an LLM with structured outputs
4. Using function calling with an LLM
5. Calling a VLM

After that we will go through a couple of exercises:
1. Adding a function for function calling
2. Building your own structured output
3. Bonus: Using grammar models with FireworksAI


### 1. Setting up dependencies

In [ ]:
!pip install fireworks-ai
# To setup the dependencies for the full demo, follow the instruction in the README

In [ ]:
### 3. Setting up dependencies
